# 회귀분석
(경제학에서는) 경제학에서 의미를 가지는 데이터를 '종속변수 (dependent variable)'로 설정하고, '종속변수'를 설명할 수 있는 '독립변수 (independent variable)'를 이용하여 변수들 간의 관계를 통계적으로 추정하는 분석방법.
- 회귀분석을 하는 이유는 '경제학적 질문'으로 부터.
- 회귀분석을 시작하려면, 질문을 분석해 볼 데이터가 있는가?
    - '독립변수'는 '종속변수'의 '경제적 질문'과 연결되어 있어야 함.
- '질문'과 '데이터'가 준비되어 있다면, OLS 분석방법을 시도해 보자.
- OLS의 강력한 장점들
    - 모델이 다음과 같을 때, $ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_K X_K + u $
    - 두가지 조건만 만족하면, 추정치를 구할 수 있음  
        - OLS.1: $ E(X'u)=0 $
        - OLS.2: $ rank E(X'X)=K $
    - 계산법이 간단함. 추정치 $\hat{\beta}$  
    $ \hat{\beta} = (X'X)^{-1}X'Y $
    - 간단한 모델링과 계산법으로 인해, 결과해석이 간단함 (사칙연산).  
    $ \hat{Y} = X\hat{\beta} $
    - 모델이 적합한지 확인 할 수 있는 방법이 존재.
        - 모델의 설명력의 존재 유무 테스트: Wald test, F-test, R-squared
        - 모델의 적합도 측정: RMSE
        - 추정치의 통계적 의미: t-statistics
    - 모델에 따라 개선방법들이 많이 연구되어 있음.

## OLS (Ordinary Least Square); 단순회귀분석

교육서비스를 제공해야 하나?

- 교육서비스와 소득에 경제학적 접근
    - 교육에 투자해야 할 이유
        - 개인의 경우 소득.
        - 정부의 경우 미래 세금 수입.
    - 얼마나 많은 교육 서비스를 제공 할 지
        - 유한한 자원; 서비스 비용의 증가.
        - 효율적인 자원분배; 복지나 인프라에 투자 가능.


1. '종속변수'와 '독립변수'의 관계를 수식으로 표현하기  
$\begin{align}
\ln({earnings}) = \beta_{intercept} + \beta_{school} {school} + \varepsilon \nonumber
\end{align}$
1. 그래프로 데이터 이해하기
![fig_3_1_2](https://github.com/SeanJSLee/Teaching_YU_DS_basic_KR/blob/main/doc/img/Figure_3_1_1.png?raw=true)
1. OLS 실행하고 분석하기
![fig_3_1_3_a](https://github.com/SeanJSLee/Teaching_YU_DS_basic_KR/blob/main/doc/img/Figure_3_1_3_a.png?raw=true)
    - 추정결과  
    $\begin{align}
    E[\ln({earnings})|{school}] = 5.835761 + 0.0674387 \cdot {school} \nonumber
    \end{align}$
        - 대학졸업자의 일주일 평균 급여: $ e^{E[ln(earning)|school=16]} = e^{5.835761 + 0.0674387 \cdot 16} = e^{6.9147802} \sim 1007.05 $
        - 고등학교 졸업자의 일주일 평균 급여: $ e^{E[ln(earning)|school=12]} = e^{5.835761 + 0.0674387 \cdot 16} = e^{6.6450254} \sim 768.95 $
        - 추정값이 의미가 있는 지 확인하기
            - 관측치가 40만개가량이며, t-statistics가 2 이상이기 떄문에, 95% 이상의 관측치들이 0보다 큰 영향을 가지고 있음. 통계적으로 의미가 있음.
            - F-test 결과도 0으로 수렴하기 때문에, 모델은 설명력이 존재.
            - Adjusted $R^2$ 0.107은 상당한 설명력을 보여줌
            <!-- - 표준오차(standard error): 표준편차랑 거의 같음 -->
<!-- ![normal dist](https://currentprotocols.onlinelibrary.wiley.com/cms/asset/d39ab523-dfbe-4589-89ab-28afcbc9e0c1/cpz1801-fig-0005-m.jpg)   -->

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy
import matplotlib.pyplot as plt

# 소수점 설정
pd.options.display.float_format = "{:,.3f}".format

# Load the dataset
df_cps = pd.read_csv("https://raw.githubusercontent.com/SeanJSLee/Teaching_YU_DS_basic_KR/main/data/Dehejia_and_Wahba_1999/data_cps78_income.csv")
df_cps

In [ ]:
y_cps,X_cps = patsy.dmatrices('ln_income_at_23 ~ edu_year', data=df_cps, return_type='dataframe')

# Fit the OLS model
model_cps = sm.OLS(y_cps,X_cps)
result_cps = model_cps.fit(cov_type='HC0')

# Print the model summary
result_cps.summary()

In [ ]:
def draw_dist(data, alpha) :
    import numpy as np
    import matplotlib.pyplot as plt
    import scipy.stats as stats
    mu = np.nanmean(data)
    sigma = np.nanstd(data)
    n = len(data[~np.isnan(data)])
    # Alpha level for two-tailed test
    # alpha = 0.05
    # 
    fig, ax = plt.subplots()
    ax.hist(data, bins=min(round(np.sqrt(n)),50), density=True, alpha=0.8)
    # Generate points on the x axis
    x = np.linspace((mu-3*sigma), (mu+3*sigma), 1000)
    # Calculate the normal distribution's PDF at these points:
    y = stats.norm.pdf(x, mu, sigma)
    ax.plot(x, y)
    # Critical values for two-tailed test at alpha level
    critical_value_right = mu + stats.norm.ppf(1 - alpha / 2) * sigma
    critical_value_left  = mu + stats.norm.ppf(alpha / 2) * sigma
    # Plot the critical values
    ax.axvline(critical_value_right, color='r', linestyle='--', label='Critical Value (Right Tail)')
    ax.axvline(critical_value_left, color='g', linestyle='--', label='Critical Value (Left Tail)')
    # Labeling the critical values
    ax.text(critical_value_right, 0.1, f'{critical_value_right:.2f}', horizontalalignment='center', color='red')
    ax.text(critical_value_left, 0.1, f'{critical_value_left:.2f}', horizontalalignment='center', color='green')
    # Fill between for the two-tail areas
    ax.fill_between(x, y, where=(x > critical_value_left) & (x < critical_value_right), 
    color='blue', alpha=0.5, label=f'{(1-alpha):.00%} data')
    print(f'Total observation:{n} \n95% observation are in [{critical_value_left:.2f}, {critical_value_right:.2f}] \n\tnumber of observation: {len(data[(data>=critical_value_left)&(data<=critical_value_right)])}, {len(data[(data>=critical_value_left)&(data<=critical_value_right)])/len(data):.2%}')
    plt.xlabel('z')
    plt.ylabel('Probability Density')
    plt.title('Standard Normal Distribution with Two-Tail Test (α={alpha})'.format(alpha=alpha))
    plt.legend()
    plt.grid(True)
    plt.show()


alpha = 0.05
draw_dist(y_cps, alpha)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import patsy
import matplotlib.pyplot as plt

# Load the dataset
df_hps = pd.read_csv("https://raw.githubusercontent.com/SeanJSLee/Teaching_YU_DS_basic_KR/main/data/KOSIS_houshold_panel_survey/data_income_kor.csv")
df_hps

In [ ]:
y,X = patsy.dmatrices('ln_income ~ edu_year', data=df_hps, return_type='dataframe')

alpha = 0.05
draw_dist(y, alpha)

In [ ]:
# Fit the OLS model
model = sm.OLS(y, X)
result = model.fit(cov_type='HC0')
result.summary()

[statsmodels.regression.linear_model.OLSResults](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.html)

In [ ]:
# 관축치
result.nobs

 * Wald test [res.wald_test()](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.wald_test.html)  
 follows k DoF Chi-squared distribution. [critical value table](https://www.itl.nist.gov/div898/handbook/eda/section3/eda3674.htm)

In [ ]:
result.wald_test(result.params.index.to_list(), use_f=False, scalar=True)

In [ ]:
from scipy.stats import chi2
chi2.ppf(.99,df=2)

In [ ]:
# 추정치
result.params

In [ ]:
# Standard Error
result.bse

In [ ]:
# t-statistic
result.tvalues

In [ ]:
# p-value
result.pvalues

In [ ]:
# confidence inverval
result.conf_int()

In [ ]:
# adjusted R-squared
result.rsquared_adj

In [ ]:
# RMSE; root mean squred error
result.mse_resid**.5

In [ ]:
# 
est_beta_intercept = result.params['Intercept']
est_beta_intercept

In [ ]:
est_beta_school = result.params['edu_year']
est_beta_school

In [ ]:
# 대학졸업자 월평균 임금
np.exp(est_beta_intercept + est_beta_school * 16)

In [ ]:
# 고등학교졸업자 월평균 임금
np.exp(est_beta_intercept + est_beta_school * 12)

$ \hat{\beta} = (X'X)^{-1}X'Y $

In [ ]:
mat_x = np.matrix(X)
mat_y = np.matrix(y)

xTx = mat_x.T * mat_x
xTx_inv = np.linalg.inv(xTx)
xTy = mat_x.T * mat_y

beta_hat = xTx_inv * xTy
print('beta :',beta_hat, '\nbeta 0:',beta_hat[0], '\nbeta 1:',beta_hat[1])